In [ ]:
from utils import *
from tqdm.notebook import tqdm
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import NearestNeighbors
from matplotlib.colors import TwoSlopeNorm

import warnings
warnings.filterwarnings("ignore")

res_1km = 1 / 12 / 10

In [2]:
def get_ar_circle(radius=10):
    radius_clip = {3: 2.54, 4:3.53, 5: 4.49, 6: 5.52, 7: 6.52, 8: 7.49, 9: 8.52, 10: 9.49, 11: 10.52, 12: 11.49, 13: 12.49, 14: 13.51, 15: 14.49, 16: 15.52, 17: 16.46, 18: 17.49, 19: 18.49, 20: 19.47, 21: 20.49, 22: 21.49, 23: 22.49, 24: 23.49, 25: 24.43}[radius]
    gdf_circle = gpd.GeoDataFrame({}, geometry=gpd.GeoDataFrame({}, geometry=gpd.points_from_xy([0], [0])).buffer(radius_clip), crs="epsg:4326")
    da_circle = xr.DataArray(np.ones((radius * 2 + 1, radius * 2 + 1)), coords={"y": np.arange(-radius, radius + 1), "x": np.arange(-radius, radius + 1)})\
        .rio.write_crs("epsg:4326")\
        .rio.clip(gdf_circle.geometry, all_touched=True, drop=False)\
        .fillna(0).astype(np.uint8)
    return da_circle, da_circle.values

In [3]:
st1 = """Value	Color	Description
1	#05450a	Evergreen Needleleaf Forests: dominated by evergreen conifer trees (canopy >2m). Tree cover >60%.
2	#086a10	Evergreen Broadleaf Forests: dominated by evergreen broadleaf and palmate trees (canopy >2m). Tree cover >60%.
3	#54a708	Deciduous Needleleaf Forests: dominated by deciduous needleleaf (larch) trees (canopy >2m). Tree cover >60%.
4	#78d203	Deciduous Broadleaf Forests: dominated by deciduous broadleaf trees (canopy >2m). Tree cover >60%.
5	#009900	Mixed Forests: dominated by neither deciduous nor evergreen (40-60% of each) tree type (canopy >2m). Tree cover >60%.
6	#c6b044	Closed Shrublands: dominated by woody perennials (1-2m height) >60% cover.
7	#dcd159	Open Shrublands: dominated by woody perennials (1-2m height) 10-60% cover.
8	#dade48	Woody Savannas: tree cover 30-60% (canopy >2m).
9	#fbff13	Savannas: tree cover 10-30% (canopy >2m).
10	#b6ff05	Grasslands: dominated by herbaceous annuals (<2m).
11	#27ff87	Permanent Wetlands: permanently inundated lands with 30-60% water cover and >10% vegetated cover.
12	#c24f44	Croplands: at least 60% of area is cultivated cropland.
13	#a5a5a5	Urban and Built-up Lands: at least 30% impervious surface area including building materials, asphalt and vehicles.
14	#ff6d4c	Cropland/Natural Vegetation Mosaics: mosaics of small-scale cultivation 40-60% with natural tree, shrub, or herbaceous vegetation.
15	#69fff8	Permanent Snow and Ice: at least 60% of area is covered by snow and ice for at least 10 months of the year.
16	#f9ffa4	Barren: at least 60% of area is non-vegetated barren (sand, rock, soil) areas with less than 10% vegetation.
17	#1c0dff	Water Bodies: at least 60% of area is covered by permanent water bodies."""

class_table = pd.read_csv(StringIO(st1), sep="\t")
class_table["Value"] = class_table["Value"].astype(int)
class_table["luc"] = class_table["Description"].str.split(":", expand=True)[0]

code2color = class_table.set_index("Value")["Color"].to_dict()

In [5]:
def get_sample_gdf(year):
    gdf_conflict_1year = gpd.read_file(path_data / f"PSM/sample_point/conflict_{year}.shp")
    gdf_miss_conflict = gpd.read_file(path_data / f"PSM/sample_point/non_conflict_{year}.shp")
    return gdf_conflict_1year, gdf_miss_conflict

In [6]:
def clip_sample(nc_file, da_circle):
    da_ = xr.open_dataarray(nc_file)
    return xr.where(da_circle==1, da_.sel(x=da_circle.x, y=da_circle.y), np.nan)
    
def load_point_and_zonal_data(year, radius):
    da_circle, ar_circle = get_ar_circle(radius)
    gdf_conflict_1year, gdf_miss_conflict = get_sample_gdf(year)
    da_conflict_lst = [clip_sample(path_data / f"PSM20km/conflict_sample/{year}_{name_}.nc", da_circle) for name_ in ["luc_lastyear", "luc_currentyear", "road_dis", "boundary_dis", "npp_lastyear", "npp_currentyear", "pop", "pop_ly"]]
    da_non_conflict_lst = [clip_sample(path_data / f"PSM20km/non_conflict_sample/{year}_{name_}.nc", da_circle) for name_ in ["luc_lastyear", "luc_currentyear", "road_dis", "boundary_dis", "npp_lastyear", "npp_currentyear", "pop", "pop_ly"]]
    return (gdf_conflict_1year, gdf_miss_conflict), (da_conflict_lst, da_non_conflict_lst)


In [7]:
# Calculate the distance to the nearest point
from sklearn.neighbors import BallTree

def get_nearest(src_points, candidates, k_neighbors=1):
    tree = BallTree(candidates, leaf_size=15, metric='haversine')
    distances, indices = tree.query(src_points, k=k_neighbors)
    distances = distances.transpose()
    indices = indices.transpose()
    closest = indices[0]
    closest_dist = distances[0]
    return (closest, closest_dist)

def nearest_neighbor(left_gdf, right_gdf, return_dist=False):
    left_geom_col = left_gdf.geometry.name
    right_geom_col = right_gdf.geometry.name
    right = right_gdf.copy().reset_index(drop=True)
    left_radians = np.array(left_gdf[left_geom_col].apply(lambda geom: (geom.x * np.pi / 180, geom.y * np.pi / 180)).to_list())
    right_radians = np.array(right[right_geom_col].apply(lambda geom: (geom.x * np.pi / 180, geom.y * np.pi / 180)).to_list())
    closest, dist = get_nearest(src_points=left_radians, candidates=right_radians)
    closest_points = right.loc[closest]
    closest_points = closest_points.reset_index(drop=True)
    if return_dist:
        earth_radius = 6371000  # meters
        closest_points['distance'] = dist * earth_radius
    return closest_points

In [8]:
def zonal_stat_sample(gdf_conflict_1year, da_conflict_lst, gdf_conflict_lastyear):
    dis_to_conflict_lastyear = nearest_neighbor(gdf_conflict_1year, gdf_conflict_lastyear, return_dist=True)

    (
        da_combine_no_coords_lastyear, da_combine_no_coords_1year, 
        da_combine_no_coords_road_dis, da_combine_no_coords_boundary_dis, 
        da_combine_no_coords_npp_lastyear, da_combine_no_coords_npp_1year,
        da_combine_no_coords_pop, da_combine_no_coords_pop_ly,
    ) = da_conflict_lst

    total_area_ly = (~np.isnan(da_combine_no_coords_lastyear)).sum(dim=["x", "y"])
    total_area_cy = (~np.isnan(da_combine_no_coords_1year)).sum(dim=["x", "y"])
    total_pop = da_combine_no_coords_pop.sum(dim=["x", "y"]).values
    total_pop_ly = da_combine_no_coords_pop_ly.sum(dim=["x", "y"]).values

    luc_ratio_ly, luc_ratio_cy = {}, {}
    for year_lc, da_lc in zip(["ly", "cy"], [da_combine_no_coords_lastyear, da_combine_no_coords_1year]):
        for luc_, luc_name in [
            [[12, 14], f"crop_{year_lc}"], 
            [[13], f"built_{year_lc}"],
            [[1, 2, 3, 4, 5],  f"forest_{year_lc}"],
            [[6, 7], f"shrubland_{year_lc}"],
            [[10],  f"grass_{year_lc}"],
        ]:
            if year_lc == "ly":
                luc_ratio_ly[luc_name] = ((da_lc.isin(luc_)).sum(dim=["x", "y"]) / total_area_ly).values
            elif year_lc == "cy":
                luc_ratio_cy[luc_name] = ((da_lc.isin(luc_)).sum(dim=["x", "y"]) / total_area_cy).values

    gdf_conflict_1year_zonal = gdf_conflict_1year.copy()\
        .assign(pop=total_pop)\
        .assign(pop_ly=total_pop_ly)\
        .assign(pop_change=total_pop-total_pop_ly)\
        .assign(dis_c_ly=dis_to_conflict_lastyear["distance"])\
        .assign(crop_ly=luc_ratio_ly["crop_ly"]).assign(crop_cy=luc_ratio_cy["crop_cy"])\
        .assign(crop_change=lambda _df: _df["crop_cy"] - _df["crop_ly"])\
        .assign(forest_ly=luc_ratio_ly["forest_ly"]).assign(forest_cy=luc_ratio_cy["forest_cy"])\
        .assign(forest_change=lambda _df: _df["forest_cy"] - _df["forest_ly"])\
        .assign(grass_ly=luc_ratio_ly["grass_ly"]).assign(grass_cy=luc_ratio_cy["grass_cy"])\
        .assign(grass_change=lambda _df: _df["grass_cy"] - _df["grass_ly"])\
        .assign(shrubland_ly=luc_ratio_ly["shrubland_ly"]).assign(shrubland_cy=luc_ratio_cy["shrubland_cy"])\
        .assign(shrubland_change=lambda _df: _df["shrubland_cy"] - _df["shrubland_ly"])\
        .assign(built_ly=luc_ratio_ly["built_ly"]).assign(built_cy=luc_ratio_cy["built_cy"])\
        .assign(dis2road=da_combine_no_coords_road_dis.mean(dim=["x", "y"]).values)\
        .assign(dis2bound=da_combine_no_coords_boundary_dis.mean(dim=["x", "y"]).values)\
        .assign(npp_cy=da_combine_no_coords_npp_1year.mean(dim=["x", "y"]).values)\
        .assign(crop_npp_ly=xr.where(da_combine_no_coords_lastyear == 12, da_combine_no_coords_npp_lastyear, np.nan).mean(dim=["x", "y"]).values)\
        .assign(crop_npp_cy=xr.where(da_combine_no_coords_1year == 12, da_combine_no_coords_npp_1year, np.nan).mean(dim=["x", "y"]).values)\
        .assign(crop_npp_change=lambda _df: _df["crop_npp_cy"] - _df["crop_npp_ly"])\
        .assign(forest_npp_ly=xr.where(da_combine_no_coords_lastyear.isin([1, 2, 3, 4, 5]), da_combine_no_coords_npp_lastyear, np.nan).mean(dim=["x", "y"]).values)\
        .assign(forest_npp_cy=xr.where(da_combine_no_coords_1year.isin([1, 2, 3, 4, 5]), da_combine_no_coords_npp_1year, np.nan).mean(dim=["x", "y"]).values)\
        .assign(forest_npp_change=lambda _df: _df["forest_npp_cy"] - _df["forest_npp_ly"])\
        .assign(grass_npp_ly=xr.where(da_combine_no_coords_lastyear.isin([10]), da_combine_no_coords_npp_lastyear, np.nan).mean(dim=["x", "y"]).values)\
        .assign(grass_npp_cy=xr.where(da_combine_no_coords_1year.isin([10]), da_combine_no_coords_npp_1year, np.nan).mean(dim=["x", "y"]).values)\
        .assign(grass_npp_change=lambda _df: _df["grass_npp_cy"] - _df["grass_npp_ly"])\

    return gdf_conflict_1year_zonal

In [9]:
def sel_sample_1country(gdf_conflict_1year_zonal, gdf_non_conflict_1year_zonal, sel_country):
    gdf_conflict_clip = gdf_conflict_1year_zonal.clip(gdf_world.query('name_long == @sel_country'))
    gdf_non_conflict_clip = gdf_non_conflict_1year_zonal.clip(gdf_world.query('name_long == @sel_country').buffer(res_1km*100))
    # df_sample = pd.concat([gdf_conflict_clip, gdf_non_conflict_clip]).query("crop_cy > 0").reset_index(drop=True)
    df_sample_sel = pd.concat([gdf_conflict_clip, gdf_non_conflict_clip]).reset_index(drop=True)
    return df_sample_sel

In [10]:
def match_conflict_to_non_con(df_sample):
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(df_sample[['pop', 'dis_c_ly', 'crop_ly', 'built_ly', 'dis2road', 'dis2bound']])
    logistic_model = LogisticRegression(solver='liblinear', random_state=0)
    logistic_model.fit(X_scaled, df_sample['c'])
    propensity_scores = logistic_model.predict_proba(X_scaled)[:, 1]

    df_sample = df_sample.assign(ps=propensity_scores)
    df_c_match = df_sample.query('c == 1')

    nn = NearestNeighbors(n_neighbors=1, metric='euclidean')
    nn.fit(propensity_scores[df_sample['c'] == 0].reshape(-1, 1)) 

    matched_indices = []
    for idx in df_sample.query('c == 1').index:
        treated_score = propensity_scores[idx]
        nearest_idx = nn.kneighbors([[treated_score]])[1][0][0]
        matched_indices.append(df_sample[df_sample['c'] == 0].index[nearest_idx])

    matched_df = df_sample.loc[matched_indices]
    df_matched = df_c_match.reset_index(drop=True).join(matched_df.reset_index(drop=True), rsuffix="_n")
    return df_matched

In [11]:
def cal_diff_from_matched(df_matched, sel_country, year):
    diff_col = ['pop', 'crop_change', 'forest_change', 'grass_change', 'crop_npp_change', 'forest_npp_change', 'grass_npp_change']
    diff_data = {}
    keep_cols = ["x", "y"]
    for col_ in keep_cols:
        diff_data[col_] = df_matched[col_]
    for col_ in diff_col:
        diff_data[col_] = df_matched[col_] - df_matched[f"{col_}_n"]
    df_diff = pd.DataFrame(diff_data)
    df_diff = df_diff.assign(country=sel_country).assign(year=year)
    return df_diff


# pipline 1year

In [12]:
def pipline_1year(year, radius):
    (gdf_conflict_1year, gdf_miss_conflict), (da_conflict_lst, da_non_conflict_lst) = load_point_and_zonal_data(year, radius)
    gdf_conflict_lastyear = gpd.read_file(path_data / f"PSM/sample_point/conflict_{year-1}.shp")
    
    gdf_conflict_1year_zonal = zonal_stat_sample(gdf_conflict_1year, da_conflict_lst, gdf_conflict_lastyear).assign(c=1)
    gdf_non_conflict_1year_zonal = zonal_stat_sample(gdf_miss_conflict, da_non_conflict_lst, gdf_conflict_lastyear).assign(c=0)

    # select 90% conflict countries
    df_conflict_count = gpd.sjoin(gdf_conflict_1year_zonal, gdf_world[['name_long', "geometry"]]).groupby("name_long")["idx"].count().sort_values(ascending=False)
    count_all = df_conflict_count.sum()
    sel_countries = df_conflict_count[(df_conflict_count.cumsum() / count_all) < 0.9].index

    diff_data = []
    matched_data = []
    for sel_country in sel_countries:
        df_sample = sel_sample_1country(gdf_conflict_1year_zonal, gdf_non_conflict_1year_zonal, sel_country)
        df_matched = match_conflict_to_non_con(df_sample).assign(country=sel_country).assign(year=year)
        df_diff = cal_diff_from_matched(df_matched, sel_country, year)
        diff_data.append(df_diff)
        matched_data.append(df_matched)

    df_matched_m = pd.concat(matched_data).reset_index(drop=True)
    df_diff_m = pd.concat(diff_data).reset_index(drop=True)
    
    return df_matched_m, df_diff_m


In [13]:
radius = 10
for year in range(2002, 2023):
    if not (path_data / f"PSM_results/diff/{radius}km_{year}.csv").exists():
        df_matched_m, df_diff_m = pipline_1year(year, radius)
        (path_data / f"PSM_results/matched").mkdir(parents=True, exist_ok=True)
        (path_data / f"PSM_results/diff").mkdir(parents=True, exist_ok=True)
        
        df_matched_m.to_csv(path_data / f"PSM_results/matched/{radius}km_{year}.csv")
        df_diff_m.to_csv(path_data / f"PSM_results/diff/{radius}km_{year}.csv")